<img src="../images/ilmudatapy-logo.png" width="350" align="center">
<br>

<center><h1>Prediksi Kelas Protein Pada Bakteri Ragi dengan Logistic Regression</h1></center>
<hr>

__Halo, Learners!__ Di notebook ini, kita akan mempraktekkan pemodelan <i>machine learning</i> untuk kasus klasifikasi dengan algoritma __Logistic Regression__. Disini kita juga akan mencoba menggunakan K-Fold Cross Validation.

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 25px">
    <ul>
        <li>
            Logistic Regression
        </li>
        <li>
            Dataset
        </li>
        <li>
            Analisis dan visualisasi data
        </li>
        <li>
            Preprocessing
            <ul>
                <li>Train test split</li>
            </ul>
        </li>
        <li>
            Modeling
            <ul>
                <li>Klasifikasi dengan Logistic Regression</li>
                <li>Evaluasi</li>
                <li>K-Fold Cross Validation</li>
            </ul>
        </li>
    </ul>
</div>

<hr>
<div class="alert alert-success" style="margin-top: 20px">
    <strong>Catatan:</strong> Untuk menjalankan kode program Python di Jupyter Notebook, klik pada <i>cell</i> yang ingin di-<i>run</i> lalu tekan <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>

<div class="alert alert-danger" style="margin-top: 20px">
    <strong>Warning!:</strong> Jika ada kode program yang <i>error</i> atau output yang dihasilkan tidak sesuai, silahkan <b>Restart & Run All</b> kernel pada bagian menu <b>Kernel</b> di menu bar Jupyter Notebook, atau <b>Restart & Clear Output</b> kernel kemudian jalankan satu per satu <i>cell</i> secara berurutan dari atas ke bawah.
</div>
<hr>

## Logistic Regression

__Logistic Regression__ merupakan variasi dari Linear Regression yang digunakan untuk dataset dengan <i>dependent variable</i> atau kolom targetnya bertipe kategori. Algoritma ini memprediksi probabilitas label kelas sebagai fungsi dari variabel independen.

Logistik regression menyesuaikan kurva berbentuk S dengan menggunakan Linear regression dan mentransformasi estimasi numerik menjadi probabilitas. Berikut ilustrasi kurva Logistic regression.

![alt text](../images/logistic-regression.jpg)
<center><i>pic source: primo.ai</i></center>

<hr>

## Dataset

Dataset yang akan digunakan adalah dataset <a href="https://datahub.io/machine-learning/yeast">Yeast</a>. Dataset ini berisi data situs lokalisasi protein pada bakteri ragi, berdasarkan beberapa uji bio-statistik.

__Attribute Information:__

* __mcg:__ McGeoch's method for signal sequence recognition.
* __gvh:__ von Heijne's method for signal sequence recognition.
* __alm:__ Score of the ALOM membrane spanning region prediction program.
* __mit:__ Score of discriminant analysis of the amino acid content of the N-terminal region (20 residues long) of mitochondrial and non-mitochondrial proteins.
* __erl:__ Presence of "HDEL" substring (thought to act as a signal for retention in the endoplasmic reticulum lumen). Binary attribute.
* __pox:__ Peroxisomal targeting signal in the C-terminus.
* __vac:__ Score of discriminant analysis of the amino acid content of vacuolar and extracellular proteins.
* __nuc:__ Score of discriminant analysis of nuclear localization signals of nuclear and non-nuclear proteins.
* __class_protein_localization:__ Predicted Attribute, Localization site of protein. ( non-numeric )

Pertama mari kita <i>import library</i> yang dibutuhkan.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Kemudian kita <i>load</i> data ke dalam dataframe Pandas dengan <code>read_csv()</code>.

In [ ]:
# Load dataset

df = pd.read_csv("../datasets/yeast.csv")
df.head(10)

Seperti yang dapat kita lihat, ada 8 kolom fitur yang berisi nilai numerik, sementara itu kolom target kelasnya berisi nilai non-numerik.

<hr>

## Analisis dan visualisasi data

Mari kita lihat jumlah data untuk tiap kelas/kategori di kolom target <code>class_protein_localization</code>.

In [ ]:
# Menghitung jumlah data pada tiap kelas/kategori

df['class_protein_localization'].value_counts()

Kita juga dapat menampilkannya dalam bentuk visual dengan <code>countplot()</code> dari Seaborn.

In [ ]:
# Menampilkan visualisasi dengan countplot Seaborn

sns.countplot(x='class_protein_localization', data=df, palette='Set2')

Untuk kolom numerik, kita dapat melihat sebaran datanya dengan histogram.

In [ ]:
# Menampilkan histogram dari kolom fitur numerik

fig, ax = plt.subplots(ncols=4, nrows=2, figsize=(16, 6)) 

# Menambahkan subplot dengan indexing
ax0 = fig.add_subplot(ax[0,0]) 
ax1 = fig.add_subplot(ax[0,1])  
ax2 = fig.add_subplot(ax[0,2])  
ax3 = fig.add_subplot(ax[0,3]) 
ax4 = fig.add_subplot(ax[1,0])  
ax5 = fig.add_subplot(ax[1,1]) 
ax6 = fig.add_subplot(ax[1,2]) 
ax7 = fig.add_subplot(ax[1,3])   

df.hist(column='mcg', bins=50, ax=ax0)
df.hist(column='gvh', bins=50, ax=ax1)
df.hist(column='alm', bins=50, ax=ax2)
df.hist(column='mit', bins=50, ax=ax3)
df.hist(column='erl', bins=50, ax=ax4)
df.hist(column='pox', bins=50, ax=ax5)
df.hist(column='vac', bins=50, ax=ax6)
df.hist(column='nuc', bins=50, ax=ax7)

plt.subplots_adjust(wspace=0.2, hspace=0.4)
plt.show()

Korelasi antar fitur dapat ditunjukkan dengan <code>heatmap()</code> dari Seaborn.

In [ ]:
# Menampilkan korelasi antar fitur

plt.figure(figsize=(12,7))

corr = df.corr()
sns.heatmap(corr, annot=True, fmt='.2f')

<hr>

## Preprocessing

Mari kita lihat <code>info()</code> dari dataframe <code>df</code>.

In [ ]:
# Melihat info dataframe

df.info()

Dari hasil di atas, tidak terlihat ada <i>missing values</i> dan tipe datanya juga sudah sesuai. Karena itu, kita dapat lanjut ke proses <i>train test split</i>.

### Train test split

Mari kita definisikan terlebih dahulu fitur dan targetnya. 

In [ ]:
# Mendefinisikan fitur dan target

X = df.drop('class_protein_localization', axis=1)       # fitur
y = df['class_protein_localization']                    # target

Selanjutnya kita lakukan <code>train_test_split()</code>.

In [ ]:
from sklearn.model_selection import train_test_split

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

<hr>

## Modeling 

### Klasifikasi dengan Logistic Regression

Dalam pemodelan dengan <code>LogisticRegression()</code> ini, kita menggunakan <code>solver = 'liblinear'</code> dan <code>class_weight = 'balanced'</code>.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Training dan testing model
model_logreg = LogisticRegression(solver='liblinear', class_weight='balanced').fit(X_train, y_train)
y_pred = model_logreg.predict(X_test)

### Evaluasi

Selanjutnya kita evaluasi kinerja model tersebut dengan <code>accuracy_score()</code>.

In [ ]:
from sklearn.metrics import accuracy_score

# Skor akurasi untuk data train dan test
print('Train set Accuracy: %.2f' % accuracy_score(y_train, model_logreg.predict(X_train)))
print('Test set Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Kita juga dapat menampilkan <code>classification_report</code>-nya.

In [ ]:
from sklearn.metrics import classification_report

print('Classification report : ')
print(classification_report(y_test, y_pred, zero_division=1))

Seperti yang dapat kita lihat, hasil akurasi untuk <i>data test</i> sekitar __0.58__, sementara untuk <i>data train</i> sekitar __0.57__.

Jika model di atas menggunakan <i>train test split</i> untuk mengevaluasi kinerja model, sekarang kita akan mencoba menggunakan K-Fold Cross Validation untuk kasus yang sama.

### K-Fold Cross Validation

__Cross Validation__ adalah sebuah teknik <i>resampling</i> yang digunakan untuk mengevaluasi model <i>machine learning</i> pada sampel data yang jumlahnya terbatas. Teknik ini memiliki parameter tunggal, yaitu <b>K</b>, yang merujuk pada jumlah kelompok yang akan dibagi menjadi sampel data. Karena itu, teknik ini biasa disebut dengan __K-Fold Cross Validation__.

Jadi, misalnya kita memilih nilai 5 sebagai K, berarti kita akan membagi dataset menjadi 5 kelompok. Nantinya akan ada 5 kali proses, dimana dalam tiap proses, salah satu kelompok menjadi <i>test data</i> dan sisanya menjadi <i>train data</i>. Skor akurasi yang dihasilkan merupakan skor rata-rata dari 5 proses tersebut.  

![alt text](images/k-fold-cross-validation.png)

Di Python, K-Fold Cross Validation dapat digunakan dengan mengimpor <i>package</i> <code>KFold</code> dari <i>library</i> <code>sklearn.model_selection</code>. Nilai K dapat didefinisikan dengan parameter <code>n_splits</code>.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=7, random_state=3, shuffle=True) 

X = df.drop('class_protein_localization', axis=1)       # fitur
y = df['class_protein_localization']                    # target

# Training dan testing model
model_logreg = LogisticRegression(solver='liblinear', class_weight='balanced').fit(X, y)

# Evaluasi model dengan cross_val_score dengan scoring 'accuracy'
scores = cross_val_score(model_logreg, X, y, scoring='accuracy', cv=kf, n_jobs=-1)
print('Accuracy score: %.2f' % (np.mean(scores)))

Dari hasil di atas, akurasi yang didapatkan adalah sebesar __0.57__.

Kita juga dapat melihat akurasi tiap iterasi seperti di bawah ini.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold 

n = 7
scores = []

kf = KFold(n_splits=n, random_state=3, shuffle=True) 

X = df.drop('class_protein_localization', axis=1)       # fitur
y = df['class_protein_localization']                    # target

for train_index, test_index in kf.split(df):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index] 
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Training dan testing model
    model_logreg = LogisticRegression(solver='liblinear', class_weight='balanced').fit(X_train, y_train)

    score_logreg = model_logreg.score(X_test, y_test)
    scores.append(score_logreg)
    display(score_logreg)
    
mean_acc = sum(scores) / n
print('Mean accuracy score : %.2f' % mean_acc)

<hr>

Copyright @ <a href="https://ilmudatapy.com/">ilmudatapy.com</a>